In [ ]:
from transformers import BertTokenizer, RobertaTokenizer
from model.engine import BertClassifier
from model.bert import BertModel
from model.roberta import RobertaModel
from train import DoubleLoss
from transformers import AutoModel, BertModel

import torch
import torch.nn as nn
import torch.nn.functional as F


# Since, we want to optimize only parameters of the adapter modules and layer normalization
# l=["adapter", "LayerNorm"]
# [n for n, p in model.named_parameters() if any([(nd in n) for nd in l])]

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = BertClassifier(model_path='roberta-base', num_labels=100, tokenizer='roberta-base')
model_bert = AutoModel.from_pretrained('roberta-base')



In [9]:
import torch
from config import *
from torch import nn
import config
import wandb
from tqdm import tqdm, trange
from model import BertClassifier
from transformers import BertTokenizer, AdamW , get_linear_schedule_with_warmup
import torch
import torch.nn.functional as F



layers = ["adapter", "LayerNorm"]
params = [p for n, p in model_bert.named_parameters() \
                if any([(nd in n) for nd in layers])]

optimizer = AdamW(params, lr=config.LEARNING_RATE)
# scheduler = get_linear_schedule_with_warmup(
#     optimizer, num_warmup_steps = int(0.1 * len(self.train_dataset))*config.EPOCHS
# )

/home/luungoc/.local/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
no_decay = ["adapter", "LayerNorm"]
# optimizer_grouped_parameters = [
#     {
#         "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#         "weight_decay": 0.1,
#     },
#     {
#         "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
#         "weight_decay": 0.0,
#     },
# ]

no_decay = ["adapter.proj_up.bias", "adapter.proj_down.bias", "LayerNorm"]
cls_bias = ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
cls_weight = ['cls.seq_relationship.weight', 'cls.predicions.transform.dense.weight', 'cls.predictions.decoder.weight']
layers = ["adapter.proj_up.weight", "adapter.proj_down.weight"]
layers.extend(cls_weight)
no_decay.extend(cls_bias)

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if any([nd in n for nd in layers])],
        "weight_decay": 0.1,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

In [24]:
trainable_params_count = 0
for group in optimizer_grouped_parameters:
    for param in group["params"]:
        trainable_params_count += param.numel()
        
trainable_params_count

2417664

In [ ]:
b1 = model_bert(tok).last_hidden_state[0, 0, :]  
b2 = torch.tensor([0 for _ in range(100)], dtype=torch.float)
b2[1] = 1

In [ ]:
def evaluate_event_detection(predictions, ground_truth):
    predicted_events = set((event[0], frozenset(event[1].items())) for event in predictions)
    actual_events = set((event[0], frozenset(event[1].items())) for event in ground_truth)

    correct_predictions = len(predicted_events.intersection(actual_events))
    total_predicted = len(predicted_events)
    total_actual = len(actual_events)

    precision = correct_predictions / total_predicted if total_predicted > 0 else 0.0
    recall = correct_predictions / total_actual if total_actual > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0

    return precision, recall, f1_score

# Ví dụ sử dụng hàm đánh giá
predicted_events = [("attack", {"location": "city", "time": "morning"}),
                     ("meeting", {"location": "office", "time": "afternoon"})]

actual_events = [("meeting", {"location": "office", "time": "afternoon"}),
                 ("celebration", {"location": "park", "time": "evening"})]

# Đánh giá
precision, recall, f1_score = evaluate_event_detection(predicted_events, actual_events)

# In kết quả
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
